In [15]:
import os
import json
import pprint
import re
from PIL import Image as PILImage
from Prompts import *
import pickle as pkl
import yaml
from utils.config import *
from utils.models import *
import copy
import subprocess
import pprint

In [16]:
model = GPTModel(config = dict(
    MODEL_NAME = MODEL
))

In [17]:
with open(os.path.join('locations',LOCATION,'scene_graph.json'),'r') as f:
    scene_graph = json.load(f)
scgraph = utils.SceneGraph(scene_graph)
encoded_img = utils.encode_image(os.path.join('locations',LOCATION,'scene_graph.png'))

In [25]:
scgraph.graph.nodes

NodeView(('11', '07', 'f7', 'd9', 'aa', '69', 'cf', '0e', '32', 'cb', 'ba', '99', 'e3', 'f5', '48', '47', 'c5', '79'))

In [30]:
('11','07') in scgraph.graph.edges

True

In [4]:
node_types = []
for node in scgraph.get_parent_nodes():
    if scgraph.graph.nodes[node]['type'] not in node_types:
        node_types.append(scgraph.graph.nodes[node]['type'])
print(node_types)

['AISLE', 'CORNER', 'INTERSECTION', 'OPEN AREA']


In [5]:
payload = [{"role": "system", 
     "content": [{
         "type":"text",
         "text":f"""
        You are an expert floor planner and a software engineer. You ALWAYS provide output that in JSON which is fully parseable with json.dumps in python.
         """}]
     },
{"role": "user", 
     "content": [{
         "type":"text",
         "text":f"""
The image shows a location which is represented by a scene graph. A scene graph is a graph with nodes (numbered red circles) representing locations and edges (blue lines) connecting them. 
A person/ can only move from one node to another if the two nodes are connected by an edge. 
The scene graph in json file format is also given below, where each node and edge has a unique id, a type, the pixel position of the node in the image (pos). 
{scene_graph}
The type of a node is one among {node_types}. 
The graph is bidirectional and each edge is also represented with the edge list 'links' in the json given below.  
Remember that a node can only be reached from another node if they have an edge between them in the scene graph json.
Now answer the following questions below:
Is node 0e connected to node 07?
Return an answer in JSON format shown below:"""+"""
{
    'ANSWER': True/False,
    'REASONING': <Reasoning for answer>
}
"""},
     {         
        "type":"image_url",
        "image_url":{
            "url":f"data:image/jpeg;base64,{encoded_img}",
            "detail":"high"
        }
     }]
     },
{"role": "assistant", "content": [{"type":"text","text":"""
{
    'ANSWER': False
    'REASONING': ""To determine if node "0e" is connected to node "07," we need to trace the paths in the provided edges.
Nodes:
"0e" at position [80, 660] (type: CORNER)
"07" at position [28, 318] (type: CORNER)
Edges involving "0e":
{"type": "PASSAGEWAY", "source": "0e", "target": "32"}
{"type": "PASSAGEWAY", "source": "0e", "target": "cb"}
Edges involving "07":
{"type": "NARROW PASSAGEWAY", "source": "11", "target": "07"}
{"type": "PASSAGEWAY", "source": "07", "target": "f7"}
There are no direct or indirect edges connecting "0e" to "07" through other nodes. Therefore, node "0e" is not connected to node "07" "".
}
"""
}]},
{"role": "user", "content": [{"type":"text","text":"""If I walk from node d9 to node aa and take a left turn which node will I reach first?
Return an answer in JSON format shown below:"""+"""
{
    'Answer': Node ID,
    'Reasoning': <Reasoning for answer>
}"""}]},
{"role": "assistant", "content": [{"type":"text",
"text":"""{
    
'ANSWER': "47",

'REASONING':"
    To determine which node you will reach first if you walk from node "d9" to node "aa" and then take a left turn, we need to look at the nodes connected to "aa" and identify the left turn from "aa".
    From the JSON structure:
    Node "aa" is connected to:
    Node "d9"
    Node "47"
    Node "69"
    Node "99"
    Since you're starting from node "d9" and reaching node "aa", taking a left turn would typically mean turning towards the next node that is positioned to the left relative to your current direction of movement. Based on the provided positions and assuming you're facing north when reaching "aa" from "d9":
    "d9" to "aa" positions: (339, 317) to (507, 319) indicates you're moving east.
    A left turn from this direction would point you north.
    Among the connected nodes to "aa", the one positioned north (left turn) is "47":
    "47" position: [503, 148]
    Therefore, if you walk from node "d9" to node "aa" and take a left turn, you will first reach node "47"."
    }"""
}]},

{"role": "user", "content": [{"type":"text","text":"""If I walk from node ba to node 69, which direction am I walking?
                              Return an answer in JSON format shown below:"""+"""
{
    'Answer': DIRECTION,
    'Reasoning': <Reasoning for answer>
}"""}]},

{"role": "assistant", "content": [{"type":"text","text":"""
                                   
{   
    'ANSWER': 'NORTH-WEST',
    'REASONING': ""To determine the direction you are walking when moving from node "ba" to node "69", we can look at the positions of these nodes in the JSON structure.
Node positions:
Node "ba": [790, 667]
Node "69": [686, 501]
Given these coordinates:
Node "ba" is located at (790, 667)
Node "69" is located at (686, 501)
Calculating the Direction:
The x-coordinate decreases from 790 to 686.
The y-coordinate decreases from 667 to 501.
Since both the x and y coordinates are decreasing, you are moving in a direction that is generally north-west.
Direction Analysis:
A decrease in the x-coordinate indicates movement towards the west.
A decrease in the y-coordinate indicates movement towards the north.
Conclusion:
When walking from node "ba" to node "69", you are walking in a north-west direction.""
}
"""}]},
{"role": "user", "content": [{"type":"text","text":"""If I walk from node 79 to node 99, which direction should I turn to reach node e3?
Return an answer in JSON format shown below:"""+"""
{
    'Answer': DIRECTION,
    'Reasoning': <Reasoning for answer>
}"""}]},
{"role": "assistant", "content": [{"type":"text","text":"""
{
    'Answer': 'left (east)',
    'REASONING': ""To determine the direction you should turn to reach node "e3" after walking from node "79" to node "99," we need to look at the relative positions of these nodes.
Node positions:
Node "79": [883, 133]
Node "99": [893, 315]
Node "e3": [1124, 309]
Movement Analysis:
Walking from node "79" to node "99":
"79" is located at (883, 133).
"99" is located at (893, 315).
Moving from "79" to "99" involves primarily moving south (as the y-coordinate increases significantly more than the x-coordinate).
Determining the direction to node "e3" from node "99":
"99" is located at (893, 315).
"e3" is located at (1124, 309).
Moving from "99" to "e3" involves primarily moving east (as the x-coordinate increases significantly and the y-coordinate remains almost the same).
Conclusion:
After walking south from "79" to "99", you should turn left (east) to reach node "e3".""
}

"""}]}]

In [6]:
scenario_1 = [{"role": "user", "content": [
    {"type":"text",
     "text":"""
Let's initialize a simulator (like Gazebo) with the world given in the image and described by the json file and try to orchestrate the following human-robot scenario: 
        
Robot is transporting a box from one point to another in the warehouse. While moving along a passageway, it encounters an employee walking in the opposite direction.

You must select the trajectories of the humans and the robot to orchestrate this scenario. You must also assign integer group ids to the HUMANS ONLY involved in the scenario according to the 
following rule:
1. All the members of the same group must have the same group id.
2. If a human is not involved in a group, they will have a group id of -1.
3. If the scenario does not specifically mention a group, then assign -1 to all humans
4. All humans involved in the scenario must be assigned a group id.
Format your output in json as given below:
        {
            'REASONING': <Reasoning for Answer>,
            'TRAJECTORIES': {
                'ROBOT': <...>
                'HUMAN 1': <...>,
                'HUMAN 2': <...>,
                'HUMAN 3': <...>,
            },
            'GROUP ID':{
                'HUMAN 1': ..,
                'HUMAN 2': ...,
                ...
            }
        }"""
}]},
{"role": "assistant", "content": [{"type":"text","text":"""
{
'REASONING':""To create a scenario where robot is transporting a box from one point to another in the warehouse and encounters an employee walking in the opposite direction along a passageway, we need to identify a suitable passageway and ensure that robot and the employee move towards each other from opposite directions.

        ### Scenario Setup:
        1. robot starts at one node and moves along a passageway towards a destination node.
        2. An employee starts at the destination node and moves along the same passageway towards robot.

        ### Suggested Passageway:
        We'll use the passageway between nodes "07", "f7", and "d9" again as it is a straightforward path that allows for clear movement in opposite directions.

        ### robot's Trajectory:
        1. **Start at node "f7"** (robot picks up the box here)
        2. **Move to node "07"** (robot moves along the passageway)

        ### Employee's Trajectory:
        1. **Start at node "07"**
        2. **Move to node "f7"** (The employee moves along the same passageway towards robot)

        ### Detailed Node Sequence:

        #### robot's Nodes:
        1. f7
        2. 07

        #### Employee's Nodes:
        1. 07
        2. f7

        ### Path Visualization:
        - **robot's Path**: f7 → 07
        - **Employee's Path**: 07 → f7

        ### Simulator Initialization:
        - **robot** starts at node "f7" and follows the path f7 → 07.
        - **Employee** starts at node "07" and follows the path 07 → f7.

        Since there is only one human in the scenario and the human is not involved in any groups, the group id for this human is the default -1.

        In a simula 

In [7]:
scenario_2 = [
{"role": "user", "content":[{"type":"text","text":"""
Now generate trajectories for the following scenario: 
                             
The robot starts at an intersection and needs to transport a box to another intersection on the other side of the warehouse. As it navigates through passageways and intersections, it reaches an open area where two warehouse employees are standing and talking. The open area is halfway through the robot's planned route.
        
You must select the trajectories of the humans and the robot to orchestrate this scenario. You must also assign integer group ids to the HUMANS ONLY involved in the scenario according to the following rule:
1. All the members of the same group must have the same group id.
2. If a human is not involved in a group, they will have a group id of -1.
3. All humans involved in the scenario must be assigned a group id.
Format your output in json as given below:
        {
           'REASONING': <Reasoning for Answer>,
             'TRAJECTORIES': {
                'ROBOT': <...>
                'HUMAN 1': <...>,
                'HUMAN 2': <...>,
                'HUMAN 3': <...>,
            },
            'GROUP IDS':{
                'HUMAN 1': ..,
                'HUMAN 2': ...,
                ...
            }
}"""
}]},
{"role": "assistant", "content": [{"type":"text","text":"""
{                                   
'REASONING':""To create the scenario where a robot starts at an intersection, transports a box to another intersection on the other side of the warehouse, and encounters two employees standing and talking in an open area halfway through its route, we need to carefully plan the robot's trajectory and identify a suitable open area for the employees.

    ### Scenario Setup:
    1. The robot starts at an intersection.
    2. The robot's destination is another intersection on the other side of the warehouse.
    3. The robot navigates through passageways and intersections.
    4. The robot reaches an open area halfway through its route where two employees are standing and talking.

    ### Selected Intersections and Open Area:
    - **Start Intersection**: Node "f7"
    - **Destination Intersection**: Node "e3"
    - **Open Area (halfway point)**: Node "69"

    ### Robot's Trajectory:
    1. **Start at node "f7"**.
    2. **Move to node "d9"** (intersection).
    3. **Move to node "aa"** (intersection).
    4. **Move to node "69"** (open area, halfway point where the employees are).
    5. **Move to node "cf"** (intersection).
    6. **Move to node "e3"** (destination intersection).

    ### Detailed Node Sequence for the Robot:
    1. f7
    2. d9
    3. aa
    4. 69
    5. cf
    6. e3

    ### Positioning the Employees:
    - Employees will be standing at node "69".
    - Since the employees, Human 1 and Human 2 form a group, they are given the same integer group id. Let's assign 1 to both their group ids.
    ### Path Visualization:
    - **Robot's Path**: f7 → d9 → aa → 69 → cf → e3

    ### Simulator Initialization:
    1. **Robot**:
    - Starts at node "f7".
    - Follows the path: f7 → d9 → aa → 69 → cf → e3.
    2. **Employees**:
    - Positioned at node "69" in the open area.
    - Standing and talking as the robot passes through.

    ### Steps to Simulate:
    1. Initialize the robot at node "f7" with a box.
    2. Set the robot to move along the path f7 → d9 → aa → 69 → cf → e3.
    3. Place the employees at node "69" and set their behavior to standing and talking.
    4. Monitor the robot's movement to ensure it encounters the employees at the open area (node "69").

    This setup ensures that the robot, while navigating from one intersection to another, encounters two employees standing and talking in an open area halfway through its planned route."",

'TRAJECTORIES': {
        'ROBOT': ['f7','d9','aa','69','cf','e3']
        'HUMAN 1': ['69'],
        'HUMAN 2':['69']
        },
'GROUP ID':{
    'HUMAN 1': 1,
    'HUMAN 2': 1
}, 
}"""}]}]

In [8]:
new_scenario_prompt = {"role": "user", "content": [{"type":"text","text":"""Now generate trajectories for the following scenario: 

The robot is tasked with transporting a box through a network of passageways, intersections, and open areas in a small warehouse. While navigating, the robot enters an open area where it encounters a group of 3 warehouse employees standing and conversing

You must select the trajectories of the humans and the robot to orchestrate this scenario. You must also assign integer group ids to the HUMANS ONLY involved in the scenario according to the following rule:
1. All the members of the same group must have the same group id.
2. If a human is not involved in a group, they will have a group id of -1.
3. If the scenario does not specifically mention a group, then assign -1 to all humans
4. All humans involved in the scenario must be assigned a group id.
Format your output in JSON as given below:
        {
            'REASONING': <Reasoning for Answer>,
            'TRAJECTORIES': {
                'ROBOT': <...>
                'HUMAN 1': <...>,
                'HUMAN 2': <...>,
                'HUMAN 3': <...>,
            },
            'GROUP IDS':{
                'HUMAN 1': ..,
                'HUMAN 2': ...,
                ...
            }
}"""}]}

In [9]:
for msg in scenario_1:
    payload.append(msg)
for msg in scenario_2:
    payload.append(msg)
payload.append(new_scenario_prompt)

In [13]:
response = model.get_response(
    model = MODEL,
    messages = payload,
    format = "json_object",
    expected_keys = ['trajectories','reasoning','group ids']
)

In [14]:
response

{'REASONING': "To create the scenario where a robot is tasked with transporting a box through a network of passageways, intersections, and open areas in a small warehouse, and encounters a group of 3 employees in an open area, we need to carefully plan the robot's trajectory and identify a suitable open area for the employees to be standing and talking. We will also assign a group ID to the humans as they form a group of 3 employees standing and conversing in the open area.",
 'TRAJECTORIES': {'ROBOT': ['f7', 'd9', 'aa', '69', 'cf'],
  'HUMAN 1': ['69'],
  'HUMAN 2': ['69'],
  'HUMAN 3': ['69']},
 'GROUP IDS': {'HUMAN 1': 1, 'HUMAN 2': 1, 'HUMAN 3': 1}}

In [ ]:
response = model.client.chat.completions.create(
    model = MODEL,
    messages=payload,
    response_format={"type": "json_object"}
    )

In [ ]:
output = json.loads(response.choices[0].message.content)
output_keys = [x.lower() for x in list(output.keys())]

with open('responses/improved_location.json','w') as f:
    json.dump(response.choices[0].message.content,f)

In [ ]:
pprint.pprint(json.loads(response.choices[0].message.content))

In [ ]:
reply = """
You made a mistake:
48 is not connected to f7. Return it in the same JSON format.
"""

In [ ]:
payload.append(
{
    "role":"assistant",
    "content":[
        {
            "type":"text",
            "text": response.choices[0].message.content
        }
    ]
}    
)
payload.append({
    "role": "user", 
    "content": [
        {
            "type":"text",
            "text": reply +"""Revise your answer. Return it in the same JSON format."""
        }
                ]
              }
               )

In [ ]:
response = model.client.chat.completions.create(
    model = MODEL,
    messages=payload,
    response_format={"type": "json_object"}
)

In [ ]:
pprint.pprint(json.loads(response.choices[0].message.content))

In [ ]:
# Hard coded for current small warehouse map scene
def pix2world(px):
    return [-1.0*((px[1]/3.0034965034965) * 0.050000 + -7.000) ,-1*((px[0]/2.6604554865424) * 0.050000 + -10.500000)]

In [ ]:
trajectories = json.loads(response.choices[0].message.content)['TRAJECTORIES']

In [ ]:
print(trajectories)

In [ ]:
pixel_trajectories = {}
for k,v in trajectories.items():
    if 'ROBOT' in k or 'HUMAN' in k:
        pixel_trajectories[k] = []
        for l in v:
            pixel_trajectories[k].append(pix2world(scgraph.graph.nodes[l]['pos']))
print(pixel_trajectories)

pix_pos_passageway ={'HUMAN 1': [(28,318), (31,149)], 'ROBOT': [(31,149), (28,318), (186,320)]}
group_open_area = {'HUMAN 1':[(686,501)],'HUMAN 2':[(686,501)],'HUMAN 3':[(686,501)],'ROBOT':[(186,320)]}
world_pos = {}

for k,v in group_open_area.items():
    world_pos[k] = []
    for l in v:
        world_pos[k].append(pix2world(l))

### BT Generation

In [1]:
btq = BTQuery()
btq_payload = btq.get_full_prompt(
    behavior = """If the robot is visible and in the way, Human 1 requests the robot to 'WAIT' so they can overtake. If the robot moves aside, Human 1 proceeds to overtake and continues to their goal. If the robot blocks the way or does not respond, Human 1 stands and watches the robot briefly before proceeding to navigate around it 'only if there is enough space."""
)

NameError: name 'BTQuery' is not defined

In [ ]:
len(btq_payload)

In [ ]:
response = model.client.chat.completions.create(
    model = MODEL,
    messages=btq_payload,
    response_format={"type": "json_object"}
    )

In [ ]:
pprint.pprint(json.loads(response.choices[0].message.content))